In [1]:
import org.apache.spark.sql.SparkSession

// Step 1: Initialize SparkSession
val spark = SparkSession.builder()
  .appName("Transfer File from GCS to HDFS")
  .getOrCreate()

spark = org.apache.spark.sql.SparkSession@76ddf55f


org.apache.spark.sql.SparkSession@76ddf55f

In [2]:
val gcsPath = "gs://spark_learning_1/notebooks/movies.csv"
val hdfsPath = "hdfs:///user/casestudies/casestudy4/movies.csv"

val data = spark.read
  .option("header", "true")  // Read the header from the CSV file
  .csv(gcsPath)

// Step 4: Write the file to HDFS with headers
data.write
  .option("header", "true")  // Include the header in the output
  .mode("overwrite")         // Overwrite if the file already exists
  .csv(hdfsPath)

println(s"Transfer of file from GCS path: $gcsPath to HDFS path: $hdfsPath has been completed successfully!")

Transfer of file from GCS path: gs://spark_learning_1/notebooks/movies.csv to HDFS path: hdfs:///user/casestudies/casestudy4/movies.csv has been completed successfully!


gcsPath = gs://spark_learning_1/notebooks/movies.csv
hdfsPath = hdfs:///user/casestudies/casestudy4/movies.csv
data = [movieId: string, title: string ... 1 more field]


[movieId: string, title: string ... 1 more field]

In [3]:
//Adding duplicates

// Step 1: Read the existing movies.csv from HDFS
val moviesPath = "hdfs:///user/casestudies/casestudy4/movies.csv"
val moviesDF = spark.read.option("header", "true").csv(moviesPath)

// Step 2: Add 1000 duplicates by appending the same DataFrame multiple times
val duplicateMoviesDF = moviesDF.limit(1000) // Take 1000 rows to duplicate
val moviesWithDuplicatesDF = moviesDF.union(duplicateMoviesDF) // Append duplicates

// Step 3: Write the updated DataFrame with duplicates back to the same HDFS path
moviesWithDuplicatesDF.write
  .option("header", "true")
  .mode("overwrite") // Overwrite the existing file
  .csv("hdfs:///user/casestudies/casestudy4/duplicated_movies.csv")

println("1000 duplicates inserted and file updated successfully!")

1000 duplicates inserted and file updated successfully!


moviesPath = hdfs:///user/casestudies/casestudy4/movies.csv
moviesDF = [movieId: string, title: string ... 1 more field]
duplicateMoviesDF = [movieId: string, title: string ... 1 more field]
moviesWithDuplicatesDF = [movieId: string, title: string ... 1 more field]


[movieId: string, title: string ... 1 more field]

In [4]:
// Add depedencies for avro format
%AddDeps org.apache.spark spark-avro_2.12 3.3.2 --transitive

Marking org.apache.spark:spark-avro_2.12:3.3.2 for download
Obtained 12 files
Marking org.apache.spark:spark-avro_2.12:3.3.2 for download
Obtained 12 files


In [5]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

// Step 1: Initialize SparkSession
val spark = SparkSession.builder()
  .appName("Movies Duplicate Cleaner")
  .getOrCreate()

// Step 2: Load movies.csv into a DataFrame from HDFS
val moviesPath = "hdfs:///user/casestudies/casestudy4/duplicated_movies.csv"
println(s"Loading data from HDFS: $moviesPath")
val moviesDF = spark.read.option("header", "true").csv(moviesPath)

// Displaying a preview of the dataset (optional, for debugging)
moviesDF.show(5)

println("Data loaded. Starting duplicate cleaning process...")

// Step 3: Convert to RDD with composite key (movieId, title)
println("Converting DataFrame to RDD and creating composite keys (movieId, title)...")
val moviesRDD = moviesDF.rdd.map(row => {
  val movieId = row.getString(row.fieldIndex("movieId"))
  val title = row.getString(row.fieldIndex("title"))
  val genres = row.getString(row.fieldIndex("genres"))
  ((movieId, title), genres) // Key: (movieId, title), Value: genres
})

// Combine genres for duplicate keys
println("Combining genres for duplicate movie entries...")
val uniqueMoviesRDD = moviesRDD.reduceByKey((genres1, genres2) => s"$genres1|$genres2")

// Transform back to (movieId, title, genres) format
println("Reformatting RDD back to DataFrame with unique movie entries...")
val cleanedMoviesRDD = uniqueMoviesRDD.map {
  case ((movieId, title), combinedGenres) => (movieId, title, combinedGenres)
}

val cleanedMoviesDF = cleanedMoviesRDD.toDF("movieId", "title", "genres")

// Step 4: Validation
println("Validation: Comparing original and deduplicated record counts...")
val originalCount = moviesDF.count()
val deduplicatedCount = cleanedMoviesDF.count()
val duplicatesRemoved = originalCount - deduplicatedCount
println(s"Original record count: $originalCount")
println(s"Deduplicated record count: $deduplicatedCount")
println(s"Duplicates removed: $duplicatesRemoved")

// Step 5: Store the final DataFrame in Avro format
val outputPath = "hdfs:///user/casestudies/casestudy4/cleaned_movies.avro"
println(s"Saving the cleaned data in Avro format to: $outputPath")
cleanedMoviesDF.write.format("avro").save(outputPath)

println("Process complete. Cleaned data has been saved.")

Loading data from HDFS: hdfs:///user/casestudies/casestudy4/duplicated_movies.csv
+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

Data loaded. Starting duplicate cleaning process...
Converting DataFrame to RDD and creating composite keys (movieId, title)...
Combining genres for duplicate movie entries...
Reformatting RDD back to DataFrame with unique movie entries...
Validation: Comparing original and deduplicated record counts...
Original record count: 88585
Deduplicated record count: 87585
Duplicates removed: 1000
Saving the cleaned data in Avro f

spark = org.apache.spark.sql.SparkSession@76ddf55f
moviesPath = hdfs:///user/casestudies/casestudy4/duplicated_movies.csv
moviesDF = [movieId: string, title: string ... 1 more field]
moviesRDD = MapPartitionsRDD[49] at map at <console>:64
uniqueMoviesRDD = ShuffledRDD[50] at reduceByKey at <console>:73
cleanedMoviesRDD = MapPartitionsRDD[51] at map at <console>:77
cleanedMoviesDF = [movieId: string, title: string ... 1 more field]


originalCount:...


[movieId: string, title: string ... 1 more field]

In [6]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._

/**
 * Method to count duplicate records in a DataFrame based on movieId and title.
 */
def countDuplicates(inputDF: DataFrame): Long = {
  // Group by 'movieId' and 'title' to count occurrences of each group
  val groupedDF = inputDF
    .groupBy("movieId", "title")
    .count()
  
  // filter groups that have more than 1 record (duplicates)
  val duplicateGroupsDF = groupedDF.filter(col("count") > 1)
  
  // Check if there are no duplicates
  if (duplicateGroupsDF.isEmpty) {
    println("No duplicate groups found.")
    return 0L
  }
  
  
  val duplicateCount = duplicateGroupsDF
    .agg(sum(col("count") - 1).alias("duplicateCount")) // Subtract 1 for each group to exclude the first record
    .collect()
    .head
    .getLong(0)
  
  duplicateCount
}


countDuplicates: (inputDF: org.apache.spark.sql.DataFrame)Long


In [7]:
countDuplicates(cleanedMoviesDF)

No duplicate groups found.


0